use Spark Dataframes to query an Amazon review dataset that contains product reviews by various customers on Amazon’s platform.

In [1]:
#import libraries
from pyspark.sql import SparkSession

In [ ]:
# https://www.kaggle.com/datasets/mojiman/bigdatafinal007?resource=download&select=Electronics_5.json

In [2]:
spark= SparkSession.builder.appName("Amazon").master('local'). getOrCreate()

22/12/16 10:30:09 WARN Utils: Your hostname, Dipanwitas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.5 instead (on interface en0)
22/12/16 10:30:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 10:30:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/16 10:30:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df= spark.read.json('Amazon-Electronics_5.json')

In [4]:
df.show(5)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|
|0528881469|[12, 15]|    1.0|I'm a professiona...|11 25, 2010| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|
|0528881469|[43, 45]|    3.0|Well, what can I ...| 09 9, 2010|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|
|0528881469| [9, 10]|    2.0|Not going to writ...|11 24, 2010|A1H8PY3QHMQQA0|Dave M. Shaw "mac...|Great grafics, PO...|    1290556800|
|0528881469|  [0, 0]|    1.0|I've had mine for...|09 29

In [5]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [6]:
# find individual count of reviewers
df.select('reviewerID').distinct().count()

192403

In [7]:
# get number of products
df.select('asin').distinct().count()

63001

In [8]:
# find product with count of reviews in desc order 
df.select('asin').groupby('asin').count().orderBy('count', ascending=False).show()

+----------+-----+
|      asin|count|
+----------+-----+
|B007WTAJTO| 4915|
|B003ES5ZUU| 4143|
|B00DR0PDNE| 3798|
|B0019EHU8G| 3435|
|B002WE6D44| 2813|
|B003ELYQGG| 2652|
|B0002L5R78| 2599|
|B009SYZ8OC| 2542|
|B00BGGDVOO| 2104|
|B002V88HFE| 2082|
|B0074BW614| 2069|
|B000LRMS66| 1960|
|B006GWO5WK| 1917|
|B000QUUFRW| 1890|
|B005FYNSPK| 1884|
|B005HMKKH4| 1866|
|B0041Q38NU| 1812|
|B00622AG6S| 1710|
|B004QK7HI8| 1581|
|B005DKZTMG| 1564|
+----------+-----+
only showing top 20 rows



In [9]:
# get count of reviews by each user

df.select('reviewerID').groupby('reviewerID').count().orderBy('count', ascending=False).show()

+--------------+-----+
|    reviewerID|count|
+--------------+-----+
|  ADLVFFE4VBT8|  431|
|A3OXHLG6DIBRW8|  407|
|  A6FIAB28IS79|  367|
| A680RUE1FDO8B|  352|
| A5JLAU2ARJ0BO|  351|
|A1ODOGXEYECQQ8|  333|
|A36K2N527TXXJN|  281|
| ARBKYIVNYWK3C|  267|
|A25C2M3QF9G7OQ|  261|
| AWPODHOB4GFWL|  260|
|A22CW0ZHY3NJH8|  255|
|A3EXWV8FNSSFL6|  250|
|A3LGT6UZL99IW1|  245|
| A38RMU1Y5TDP9|  244|
|A2NOW4U7W3F7RI|  241|
|A23GFTVIETX7DS|  241|
|A3AYSYSLHU26U9|  231|
|A2AY4YUOX2N1BQ|  228|
|A17BUUBOU0598B|  228|
|A1UQBFCERIP7VJ|  228|
+--------------+-----+
only showing top 20 rows



In [10]:
# number of users every month sorted in descending order 
from pyspark.sql.functions import *
df= df.withColumn('ReviewedTime', to_timestamp(df['unixReviewTime']))

In [11]:
df.show(5)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|       ReviewedTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|2013-06-02 05:30:00|
|0528881469|[12, 15]|    1.0|I'm a professiona...|11 25, 2010| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|2010-11-25 05:30:00|
|0528881469|[43, 45]|    3.0|Well, what can I ...| 09 9, 2010|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|2010-09-09 05:30:00|
|0528881469| [9, 10]|    2.0|Not going to writ...|11 24, 2010|A1H8PY3Q

In [21]:
df= df.withColumn('year', year('ReviewedTime'))\
    .withColumn('month', month('ReviewedTime'))

In [22]:

df.select('year','reviewerID' ).groupby('year','reviewerID' ).count().orderBy('count', ascending=False).show()

+----+--------------+-----+
|year|    reviewerID|count|
+----+--------------+-----+
|2012|A2AY4YUOX2N1BQ|  126|
|2012| AEJAGHLC675A7|  109|
|2013|A2KYRPDKVBI2R4|  108|
|2013|A3OA4DV4L81N1D|  106|
|2013|A2HRHF83I3NDGT|  103|
|2014|A2NYK9KWFMJV4Y|   96|
|2013|  ADLVFFE4VBT8|   95|
|2013|A12DQZKRKTNF5E|   94|
|2013| A1E1LEVQ9VQNK|   93|
|2013|A1ODOGXEYECQQ8|   92|
|2013|A36K2N527TXXJN|   92|
|2014|A2BYV7S1QP2YIG|   92|
|2013|A27M75LRSJ788H|   89|
|2014|A3H4S6YO9VNGDZ|   88|
|2013| ARBKYIVNYWK3C|   87|
|2014|A1G650TTTHEAL5|   86|
|2010|A3OXHLG6DIBRW8|   85|
|2013|A3IOQ7NVVLAUEE|   85|
|2014| AAQ9NJ480N9W5|   83|
|2013|A22CW0ZHY3NJH8|   83|
+----+--------------+-----+
only showing top 20 rows



In [25]:
# how many times a product is reviewed 'great'
great_reviews= df.filter(df.reviewText.contains('great'))
great_reviews.select('asin').groupBy('asin').count().orderBy('count', ascending=False).show()

+----------+-----+
|      asin|count|
+----------+-----+
|B007WTAJTO| 1096|
|B003ES5ZUU| 1013|
|B00DR0PDNE| 1012|
|B0019EHU8G|  977|
|B003ELYQGG|  832|
|B0074BW614|  712|
|B002WE6D44|  621|
|B0002L5R78|  616|
|B00BGGDVOO|  604|
|B009SYZ8OC|  558|
|B00007E7JU|  542|
|B005DKZTMG|  536|
|B002MAPRYU|  464|
|B0027VT6V4|  440|
|B005FYNSPK|  418|
|B005HMKKH4|  413|
|B002V88HFE|  406|
|B0052SCU8U|  395|
|B000LRMS66|  392|
|B0052YFYFK|  387|
+----------+-----+
only showing top 20 rows

